In [ ]:
import os
import json
import boto3
import logging

# Initialize logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Fetch inference profile ARN from environment variables
bedrock_inference_profile_arn = os.getenv("BEDROCK_INFERENCE_PROFILE_ARN")

# Initialize the Bedrock Runtime client
bedrock_runtime = boto3.client(
    service_name="bedrock-runtime",
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY"),
    region_name=os.getenv("AWS_REGION")
)

def invoke_bedrock_model(profile_arn, payload):
    """
    Invoke a Bedrock model using the inference profile ARN.

    Args:
        profile_arn (str): ARN of the Bedrock inference profile.
        payload (dict): Input payload for the model.

    Returns:
        dict: Response from the Bedrock model.
    """
    try:
        logger.info(f"Invoking Bedrock model")
        
        response = bedrock_runtime.invoke_model(
            modelId=profile_arn,
            body=json.dumps(payload),
            contentType="application/json",
            accept="application/json"
        )
        
        # Parse and return the response
        response_body = json.loads(response["body"].read())
        return response_body

    except Exception as e:
        logger.error(f"Error invoking model with ARN {profile_arn}: {e}")
        return {"error": str(e)}

# Payload for the smart home agent's winter preparation task
smart_home_payload = {
    "prompt": """Prepare the house for winter by optimizing thermostat, plumbing, lighting, garage, and garden. 
Ensure pipes won't freeze and all systems are secure.""",
    "temperature": 0.7,
    "top_p": 0.9
}

# Call the Bedrock model
try:
    smart_home_response = invoke_bedrock_model(bedrock_inference_profile_arn, smart_home_payload)
    
    # Print the raw response
    print("Smart Home Agent Response:")
    print(json.dumps(smart_home_response, indent=2))

    # Extract and print generated instructions
    generated_text = smart_home_response.get("generation", "No generation found")
    print("\nGenerated Instructions:\n", generated_text)

except Exception as e:
    print(f"Error: {e}")

In [ ]:
# Import required libraries
import os
import boto3
import json
import logging

# Initialize Logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Fetch environment variables
agent_id = os.getenv("AWS_BEDROCK_SMART_HOME_AGENT_ID")
alias_id = os.getenv("AWS_BEDROCK_SMART_HOME_AGENT_ALIAS_ID")
region = os.getenv("AWS_REGION")

# Validate environment variables
if not agent_id:
    raise ValueError("Environment variable AWS_BEDROCK_SMART_HOME_AGENT_ID must be set.")
if not alias_id:
    raise ValueError("Environment variable AWS_BEDROCK_SMART_HOME_AGENT_ALIAS_ID must be set.")
if not region:
    raise ValueError("Environment variable AWS_REGION must be set.")

# Initialize Bedrock Agent Runtime Client
bedrock_agent_runtime_client = boto3.client(
    "bedrock-agent-runtime",
    region_ncame=region
)

# Log setup details (avoid logging sensitive data like credentials)
logger.info("Initialized Bedrock Agent Runtime Client with environment configurations successfully.")

In [ ]:
# Define the payload for the Bedrock inference model
smart_home_payload = {
    "prompt": """
    You are an AI assistant for smart home automation. 
    Help the user prepare their house for winter by providing specific, actionable instructions to optimize:
    1. Thermostat (to prevent pipes from freezing)
    2. Lighting

    Provide these instructions in a simple, step-by-step format.
    """,
    "temperature": 0.7,
    "top_p": 0.9
}

# Call the Bedrock inference model
try:
    # Ensure the inference profile ARN is set
    if not bedrock_inference_profile_arn:
        raise ValueError("Environment variable BEDROCK_INFERENCE_PROFILE_ARN must be set.")

    smart_home_response = invoke_bedrock_model(bedrock_inference_profile_arn, smart_home_payload)

    # Print the raw response
    print("Smart Home AI Assistant Response:")
    print(json.dumps(smart_home_response, indent=2))

    # Extract and display the generated instructions
    generated_text = smart_home_response.get("generation", "No generation found")
    print("\nGenerated Instructions:\n", generated_text)

    # Optionally, invoke the action groups if needed
    if "thermostat" in generated_text.lower():
        print("\nPreparing to adjust the thermostat as per instructions...")
        # Placeholder for invoking thermostat-specific actions
        # Invoke actions through Bedrock Agent Runtime or specific APIs

except Exception as e:
    print(f"Error during winter preparation task: {e}")

In [ ]:
# Parse AI instructions and interact with the Bedrock agent runtime
try:
    # Extract actionable instructions from the AI response
    ai_instructions = smart_home_response.get("generation", "")
    print("Generated AI Instructions:\n", ai_instructions)

    # Simulate parsing instructions (e.g., thermostat-related actions)
    if "thermostat" in ai_instructions.lower():
        # Define session ID for Bedrock Agent
        import uuid
        session_id = str(uuid.uuid4())

        # Create a thermostat-specific action
        action_payload = {
            "actionGroup": "action_group_thermostat",
            "function": "set_temperature",
            "parameters": {
                "temperature": 68,  # Example: Set thermostat to 68°F
                "mode": "consistent"  # Example mode for consistency
            }
        }

        # Invoke Bedrock Agent Runtime
        agent_response = bedrock_agent_runtime_client.invoke_agent(
            agentId=os.getenv("AWS_BEDROCK_SMART_HOME_AGENT_ID"),
            agentAliasId=os.getenv("AWS_BEDROCK_SMART_HOME_AGENT_ALIAS_ID"),
            sessionId=session_id,
            inputText=f"Adjust thermostat as per the instructions: {action_payload}",
        )

        # Print agent response
        print("\nAgent Response for Thermostat Action:")
        # print(agent_response) #uncomment for debugging and has sensitive data such as session ID

except Exception as e:
    print(f"Error processing and invoking agent actions: {e}")

In [ ]:
import boto3
import json
import logging

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Initialize Bedrock Agent Client
region = os.getenv("AWS_REGION")
bedrock_agent_client = boto3.client("bedrock-agent", region_name=region)

def list_agents():
    """
    List all agents in the Bedrock service.
    """
    try:
        logger.info("Listing all Bedrock agents...")
        response = bedrock_agent_client.list_agents()

        # Process the response
        agents = response.get("agents", [])
        if agents:
            logger.info(f"Found {len(agents)} agents:")
            for agent in agents:
                print(json.dumps(agent, indent=2))
        else:
            logger.info("No agents found.")
    except Exception as e:
        logger.error(f"Error listing agents: {e}")

# Call the function
list_agents()

In [ ]:
import json
import boto3
import logging

# Initialize logging
logging.basicConfig(level=logging.DEBUG)
boto3.set_stream_logger('', logging.DEBUG)
logger = logging.getLogger(__name__)

# Environment Variables
agent_id = os.getenv("AWS_BEDROCK_SMART_HOME_AGENT_ID")
alias_id = os.getenv("AWS_BEDROCK_SMART_HOME_AGENT_ALIAS_ID")
region = os.getenv("AWS_REGION")

# Initialize Bedrock Agent Runtime Client
bedrock_agent_runtime = boto3.client("bedrock-agent-runtime", region_name=region)

def invoke_agent_with_roc(prompt_text, session_id=None):
    """
    Invokes the Bedrock Agent with a prompt and handles Return of Control events.
    """
    try:
        # Send request to the agent
        logger.info(f"Invoking agent with prompt: {prompt_text}")
        response = bedrock_agent_runtime.invoke_agent(
            agentId=agent_id,
            agentAliasId=alias_id,
            sessionId=session_id or "roc-session",
            inputText=prompt_text,
            enableTrace=True
        )

        # Extract event stream
        event_stream = response.get("completion", [])
        for event in event_stream:
            if "chunk" in event:
                print(f"Agent Response: {event['chunk']['bytes'].decode()}")
            elif "returnControl" in event:
                return_control = event["returnControl"]
                logger.info(f"Return Control Event: {json.dumps(return_control, indent=2)}")

                # Simulate follow-up handling
                print("The agent returned control for action. Here are the details:")
                for invocation in return_control.get("invocationInputs", []):
                    action_group = invocation["functionInvocationInput"]["actionGroup"]
                    function = invocation["functionInvocationInput"]["function"]
                    parameters = invocation["functionInvocationInput"]["parameters"]
                    print(f"Action Group: {action_group}, Function: {function}, Parameters: {parameters}")

    except Exception as e:
        logger.error(f"Error during agent invocation: {e}")
        print({"error": str(e)})

# Prompt the agent with an unsupported request
invoke_agent_with_roc("Can you book air travel for tonight?")